In [26]:
import sys
print(sys.version)
%load_ext autoreload
%autoreload 2

3.11.8 (main, Feb 26 2024, 15:36:12) [Clang 14.0.6 ]
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from sae_lens.toolkit.pretrained_saes import get_gpt2_res_jb_saes

saes, sparsities = get_gpt2_res_jb_saes(device='mps')

In [34]:
from sae_lens.training.evals import run_evals
from sae_lens.training.activations_store import ActivationsStore
import transformer_lens
from tests.unit.helpers import build_sae_cfg
from datasets import Dataset
import rich

model = transformer_lens.HookedTransformer.from_pretrained("gpt2-small", device="mps")
cfg = build_sae_cfg(
        checkpoint_path="./checkpoints",
        train_batch_size=768,
        total_training_tokens=100000,
        context_size=64,
        device="mps"
    )
dataset = Dataset.from_list([{"text": "hello world"}] * 100000)
activation_store = ActivationsStore.from_config(model, cfg, dataset=dataset)
# for name, sae in saes.items():
name = "potato"
sae = list(saes.values())[0]
res = run_evals(sae, activation_store, model, 1000, f"_{name}")
rich.print(res)

Loaded pretrained model gpt2-small into HookedTransformer
Run name: 128-L1-0.002-LR-0.0002-Tokens-1.000e+06
n_tokens_per_buffer (millions): 4.8e-05
Lower bound: n_contexts_per_buffer (millions): 8e-06
Total training steps: 250000
Total wandb updates: 25000
n_tokens_per_feature_sampling_window (millions): 0.0012
n_tokens_per_dead_feature_window (millions): 0.024
We will reset the sparsity calculation 5000 times.
Number tokens in sparsity calculation window: 2.00e+02
torch.Size([4, 64, 50257])
torch.Size([4, 64, 50257])
tensor([[[18.6924, 28.0045, 10.5233,  ..., -9.3606, -6.9649, 22.6572],
         [18.6924, 28.0045, 10.5233,  ..., -9.3606, -6.9649, 22.6572],
         [18.6924, 28.0045, 10.5233,  ..., -9.3606, -6.9649, 22.6572],
         ...,
         [18.6924, 28.0045, 10.5233,  ..., -9.3607, -6.9649, 22.6572],
         [18.6924, 28.0045, 10.5233,  ..., -9.3607, -6.9649, 22.6572],
         [18.6924, 28.0045, 10.5233,  ..., -9.3607, -6.9649, 22.6572]],

        [[18.6924, 28.0045, 10.523

{
    'metrics/l0_norm_potato': 768.0,
    'metrics/l0_ratio_potato': 1.0,
    'metrics/l1_norm_potato': 86.08262634277344,
    'metrics/l1_ratio_potato': 0.9125170111656189,
    'metrics/l2_norm_potato': 4.857132911682129,
    'metrics/l2_ratio_potato': 0.9421585202217102,
    'metrics/CE_loss_score_potato': 1.0109683275222778,
    'metrics/ce_loss_without_sae_potato': 5.607950210571289,
    'metrics/ce_loss_with_sae_potato': 5.397302150726318,
    'metrics/ce_loss_with_ablation_potato': 24.81311798095703,
    'metrics/kl_div_potato': inf
}

In [36]:
sparse_autoencoder = sae
hook_point = sparse_autoencoder.cfg.hook_point
hook_point_layer = sparse_autoencoder.hook_point_layer
hook_point_head_index = sparse_autoencoder.cfg.hook_point_head_index
hook_point_eval = sparse_autoencoder.cfg.hook_point_eval.format(
    layer=hook_point_layer
)
### Evals
eval_tokens = activation_store.get_batch_tokens()
eval_tokens.shape

torch.Size([4, 64])

<generator object Module.parameters at 0x2c9017840>